In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from dataclasses import dataclass
from collections import deque
import json

import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch import optim
from tqdm import tqdm
import pytorch_lightning as pl
from torchvision.ops import sigmoid_focal_loss, batched_nms
from torchvision.utils import draw_bounding_boxes
from PIL import Image
from pycocotools.cocoeval import COCOeval

from modules.utils import convert_to_xywh, convert_to_xyxy, generate_subset, calc_iou, collate_fn
from modules.datasets import CocoDetection
import modules.transforms as T
from modules.models import RetinaNet
from modules.model_orig import RetinaNet as RetinaNetOrig


In [12]:
from retinanet import post_process, get_loader, Config, loss_fn


In [6]:
config = Config()
train_loader, val_loader = get_loader(config.img_dir, config.annot_file, 1)


loading annotations into memory...
Done (t=1.31s)
creating index...
index created!
loading annotations into memory...
Done (t=1.32s)
creating index...
index created!


In [23]:
model = RetinaNet(2)
model_orig = RetinaNetOrig(2)
model_orig.backbone.load_state_dict(torch.hub.load_state_dict_from_url('https://download.pytorch.org/models/resnet18-5c106cde.pth'), strict=False)


_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc.weight', 'fc.bias'])

In [24]:
img, target = next(iter(train_loader))
out = model(img)
out_orig = model_orig(img)


In [37]:
out[0].shape, out[1].shape, out[2].shape


(torch.Size([1, 65295, 2]), torch.Size([1, 65295, 4]), torch.Size([65295, 4]))

In [38]:
out_orig[0].shape, out_orig[1].shape, out_orig[2].shape


(torch.Size([1, 65295, 2]), torch.Size([1, 65295, 4]), torch.Size([65295, 4]))

In [27]:
loss_fn(out[0], out[1], out[2], target)


(tensor(1.1287, grad_fn=<DivBackward0>),
 tensor(0.1166, grad_fn=<DivBackward0>))

In [28]:
loss_fn(out_orig[0], out_orig[1], out_orig[2], target)


(tensor(1.1287, grad_fn=<DivBackward0>),
 tensor(0.1166, grad_fn=<DivBackward0>))

In [29]:
loss_func(out[0], out[1], out[2], target)


(tensor(1.1287, grad_fn=<DivBackward0>),
 tensor(0.1166, grad_fn=<DivBackward0>))

In [30]:
loss_func(out_orig[0], out_orig[1], out_orig[2], target)


(tensor(1.1287, grad_fn=<DivBackward0>),
 tensor(0.1166, grad_fn=<DivBackward0>))

In [19]:
def loss_func(preds_class: torch.Tensor, preds_box: torch.Tensor,
              anchors: torch.Tensor, targets: dict,
              iou_lower_threshold: float=0.4,
              iou_upper_threshold: float=0.5):
    anchors_xywh = convert_to_xywh(anchors)

    # 画像毎に目的関数を計算
    loss_class = preds_class.new_tensor(0)
    loss_box = preds_class.new_tensor(0)
    for img_preds_class, img_preds_box, img_targets in zip(
            preds_class, preds_box, targets):
        # 現在の画像に対する正解矩形がないとき
        if img_targets['classes'].shape[0] == 0:
            # 全ての物体クラスの確率が0となるように
            # (背景として分類されるように)ラベルを作成
            targets_class = torch.zeros_like(img_preds_class)
            loss_class += sigmoid_focal_loss(
                img_preds_class, targets_class, reduction='sum')

            continue

        # 各画素のアンカーボックスと正解矩形のIoUを計算し、
        # 各アンカーボックスに対して最大のIoUを持つ正解矩形を抽出
        ious = calc_iou(anchors, img_targets['boxes'])[0]
        ious_max, ious_argmax = ious.max(dim=1)

        # 分類のラベルを-1で初期化
        # IoUが下の閾値と上の閾値の間にあるアンカーボックスは
        # ラベルを-1として損失を計算しないようにする
        targets_class = torch.full_like(img_preds_class, -1)

        # アンカーボックスとマッチした正解矩形のIoUが下の閾値より
        # 小さい場合、全ての物体クラスの確率が0となるようラベルを用意
        targets_class[ious_max < iou_lower_threshold] = 0

        # アンカーボックスとマッチした正解矩形のIoUが上の閾値より
        # 大きい場合、陽性のアンカーボックスとして分類回帰の対象にする
        positive_masks = ious_max > iou_upper_threshold
        num_positive_anchors = positive_masks.sum()

        # 陽性のアンカーボックスについて、マッチした正解矩形が示す
        # 物体クラスの確率を1、それ以外を0として出力するように
        # ラベルに値を代入
        targets_class[positive_masks] = 0
        assigned_classes = img_targets['classes'][ious_argmax]
        targets_class[positive_masks,
                      assigned_classes[positive_masks]] = 1

        # IoUが下の閾値と上の閾値の間にあるアンカーボックスについては
        # 分類の損失を計算しない
        loss_class += ((targets_class != -1) * sigmoid_focal_loss(
            img_preds_class, targets_class)).sum() / \
            num_positive_anchors.clamp(min=1)

        # 陽性のアンカーボックスが一つも存在しないとき
        # 矩形の誤差の学習はしない
        if num_positive_anchors == 0:
            continue

        # 各アンカーボックスにマッチした正解矩形を抽出
        assigned_boxes = img_targets['boxes'][ious_argmax]
        assigned_boxes_xywh = convert_to_xywh(assigned_boxes)

        # アンカーボックスとマッチした正解矩形との誤差を計算し、
        # ラベルを作成
        targets_box = torch.zeros_like(img_preds_box)
        # 中心位置の誤差はアンカーボックスの大きさでスケール
        targets_box[:, :2] = (
            assigned_boxes_xywh[:, :2] - anchors_xywh[:, :2]) / \
            anchors_xywh[:, 2:]
        # 大きさはアンカーボックスに対するスケールのlogを予測
        targets_box[:, 2:] = (assigned_boxes_xywh[:, 2:] / \
                              anchors_xywh[:, 2:]).log()

        # L1誤差とL2誤差を組み合わせたsmooth L1誤差を使用
        loss_box += F.smooth_l1_loss(img_preds_box[positive_masks],
                                     targets_box[positive_masks],
                                     beta=1 / 9)

    batch_size = preds_class.shape[0]
    loss_class = loss_class / batch_size
    loss_box = loss_box / batch_size

    return loss_class, loss_box


In [16]:
out[2].shape


torch.Size([147312, 4])

In [17]:
out[1].shape


torch.Size([32, 147312, 4])

In [18]:
out[0].shape


torch.Size([32, 147312, 2])

In [33]:
import timm
model =timm.create_model("resnet18", features_only=True)


In [36]:
model.feature_info.channels()[-3:]


[128, 256, 512]